In [26]:
## Standard

import os
import sys
sys.path.append('../RD-systems-and-test-benches')

import math
from datetime import datetime

## Specific

from tqdm.notebook import tqdm
import copy
# from concurrent.futures import ProcessPoolExecutor
# import networkx as nx
# from itertools import product
# from itertools import chain

## Data handling 

import pandas as pd
import numpy as np
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows

import sklearn.metrics
import scipy.constants as scc
import seaborn as sns
import scipy.integrate as integrate
import scipy.optimize as sco

import pickle

## Plot

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scienceplots
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
plt.style.use(['science'])

import plotly.graph_objects as go
import plotly.io as pio

from IPython.core.display import HTML

## Own modumes

import model as ty
import heat_transfer as bht
import proc as pr

import plot_functions_here as pfun
import general as gen

import thermal.SK_standard as nm
import utils.plot_functions as RD_pfun
import repo_config as rconfig

import utils.data_processing as dp

## Physics

import fluids as fds
import ht 

In [109]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import FuncFormatter
import scienceplots
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
plt.style.use(['science'])

def to_percent(y, _):
    return f"{y * 100:.0f}\%"

In [28]:
folder_path = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-perf-1Dmodel'
geometry_path = os.path.join(folder_path, 'Panels\\Panel_V4.1NI_harp_calibrated.xlsx')
hypotheses_path = folder_path + '\\' + 'Hypotheses\\Model_hypotheses_calibrated.xlsx'
condi_path = folder_path + '\\' + 'Steady-state conditions inputs\\ssCforStudy.xlsx'
# exp_path = folder_path + '\\' + 'Experiments\\Experiments-GMI.xlsx'

panelSpecs = pr.import_geometry(geometry_path)

# panelSpecs['part2']['lambd_abs_ins'] = 0.
# panelSpecs['part2']['lambd_tube_ins'] = 0.

hyp = pr.create_dict_from_excel(hypotheses_path,'Main')

# steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre en compte
# stepConditions = steadyStateConditions_df.to_dict(orient='records')

recap = pd.read_excel(r'G:\Drive partagés\Cercle Hard\R&D\Thermal performance tests PVT\Recap_RD_TUV_SSI.xlsx')
recap = recap.drop(index=0)
recap.dropna(how='all', inplace=True)

groups = {key: group for key, group in recap.groupby('Design')}

# steadyStateConditions_df = groups['V4.1NI'].loc[groups['V4.1NI']['Type'] == 'dark'].copy()
steadyStateConditions_df = pd.read_excel(condi_path,header=0)
steadyStateConditions_df.drop(0, inplace=True)

pr.adjust_steadyStateConditions(steadyStateConditions_df, hyp, convert_celsius=True)
steadyStateConditions_df.reset_index(drop=True,inplace=True)

# for key, group in groups.items():
#     pr.convert_to_K(group)

In [ ]:
panelSpecs['part2']['lambd_tube_ins']

In [30]:
dic_names = {'T_in' : 'T_fluid_in',
             'T_out' : 'T_fluid_out',
             'Qdot_1_AG' : 'Qdot/AG',
             'T_amb' : 'T_amb',
             'u' : 'u'}

In [31]:
# df_res, list_res = ty.simu_steadyStateConditions(panelSpecs,hyp,steadyStateConditions_df)

In [ ]:
panelSpecs['part2']['delta']

In [34]:
# panelSpecs_temp = copy.deepcopy(panelSpecs)
# hyp_temp = copy.deepcopy(hyp)
# steadyStateConditions_df_temp = copy.deepcopy(steadyStateConditions_df)

In [35]:
# for el in [panelSpecs_temp[key] for key,value in panelSpecs_temp['decomp'].items()]:
#     print(el['geometry'])

In [36]:
storage_fp = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-perf-1Dmodel\StudyForThesis'

### MEANDER - Number of tubes

In [ ]:
N_range = np.arange(4,35,1)

In [ ]:
meander_dict = {}

for i,N in tqdm(enumerate(N_range), total=len(N_range)):

    panelSpecs_local = copy.deepcopy(panelSpecs)
    hyp_local = copy.deepcopy(hyp)
    steadyStateConditions_df_local = copy.deepcopy(steadyStateConditions_df)

    panelSpecs_local['part2']['N_meander'] = N

    pr.update_panelSpecs_N_tubes(panelSpecs_local)
    pr.update_panelSpecs_after_modif(panelSpecs_local)

    df_res, list_res = ty.simu_steadyStateConditions(panelSpecs_local, hyp_local, steadyStateConditions_df_local)

    tab_mat,tab_res,X = nm.find_SK(df_res,dic_names,columns_list=[0,1,0,1,0,0,0,0,0],options=[],IAM=False,LSM='nnls')

    meander_dict[N] = {'df_res' : copy.deepcopy(df_res), 'list_res' : copy.deepcopy(list_res), 'X' : copy.deepcopy(X)}

In [41]:
Ntubes_dict = dp.load_pickle(storage_fp, "V4.NI_meander_calibrated_ssCforStudy_Ntubes_dfresXlists")

### HARP - Number of tubes

In [ ]:
Dtube_range = np.array([0.001,0.003,0.005,0.01,0.02,0.03])

harp_dict = {}

# panelSpecs['part2']['e_tube'] = 0.001

for _, Dtube in enumerate(Dtube_range):

    print(Dtube)

    N_max = panelSpecs['part2']['w_abs']/(Dtube + 2*panelSpecs['part2']['e_tube'])
    N_range = np.arange(4,N_max,8)

    for i, N in tqdm(enumerate(N_range), total=len(N_range)):

        panelSpecs_local = copy.deepcopy(panelSpecs)
        hyp_local = copy.deepcopy(hyp)
        steadyStateConditions_df_local = copy.deepcopy(steadyStateConditions_df)

        panelSpecs_local['part2']['H_tube'] = Dtube
        panelSpecs_local['part2']['w_tube'] = Dtube
        panelSpecs_local['part2']['N_harp'] = N

        panelSpecs_local['part2']['update_tube_perimeter_for_convection'] = 1

        pr.update_panelSpecs_N_tubes(panelSpecs_local)
        pr.update_panelSpecs_after_modif(panelSpecs_local)

        df_res, list_res = ty.simu_steadyStateConditions(panelSpecs_local, hyp_local, steadyStateConditions_df_local)

        tab_mat,tab_res,X = nm.find_SK(df_res,dic_names,columns_list=[0,1,0,1,0,0,0,0,0],options=[],IAM=False,LSM='nnls')

        harp_dict[(Dtube,N)] = {'panelSpecs' : copy.deepcopy(panelSpecs_local),
                                 'df_res' : copy.deepcopy(df_res),
                                 'list_res' : copy.deepcopy(list_res),
                                 'X' : copy.deepcopy(X)}

In [ ]:
harp_dict[(0.001, 200)]['df_res'][['T_fluid_in','T_fluid_out','Qdot/AG','T_amb','u']]

In [ ]:
harp_dict[(0.01, 36)]['panelSpecs']['part2']['p_ext_tube']

In [ ]:
harp_dict[(0.001, 204)]['panelSpecs']['part2']['p_ext_tube']

### HARP - PLOT OF B1 VS. NUMBER OF TUBES FOR DIFFERENT DIAMETERS

In [ ]:
Dtube_range

In [ ]:
colors = ['olive', 'green', 'blue', 'orange', 'red', 'grey']
colors_lighter = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd', 'X', 'v', '^']

# Main plot: plotting L_f0_range on x-axis
fig, ax1 = plt.subplots(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))

# Set the new x-axis values for the number of tubes (assuming you have this data)


# Create the secondary Y-axis (ax2) on the right
ax2 = ax1.twinx()

X_list = Ntubes_dict['X_list']
X0 = [X_list[i][0] for i in range(len(X_list))]
meander_b1 = [X0[i][1] - 3*X0[i][3]  for i in range(len(X0))]

b1_transformed = (np.array(meander_b1) - y_ref)/y_ref
N_range = np.arange(4, 35, 1)
ax2.plot(N_range, b1_transformed, label=' ', linewidth=0.)
# ax1.plot(N_range, [X0[i][1] - 3*X0[i][3]  for i in range(len(X0))], label = 'No fin', color=colors[0], linestyle=linestyles[0], linewidth = 1.5)

ax2.set_ylabel('Relative increase in $b_{1}$ vs. reference')

for _, Dtube in enumerate(Dtube_range):

    N_max = panelSpecs['part2']['w_abs']/(Dtube + 2*panelSpecs['part2']['e_tube'])
    N_range = np.arange(4,N_max,8)

    harp_b1_list = [harp_dict[(Dtube,N)]['X'][0][1] - 3*harp_dict[(Dtube,N)]['X'][0][3] for N in N_range]

    ax1.scatter(N_range, harp_b1_list, label=f'Dtube = {Dtube} m', color=colors[_], marker=markers[_])

ax1.set_xlabel('Number of tubes')
ax1.set_ylabel(r"$b_{1} = A_1(0) \quad [W / m^{2} K]$")

ax1.grid(axis='x', linewidth=0.5, color=rconfig.get_hex_code('grey', 70))
ax2.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

# Combine the legends of both axes
lines1, labels1 = ax1.get_legend_handles_labels()
# lines3, labels3 = ax3.get_legend_handles_labels()

# Plot combined legend below the x-axis
fig.legend(lines1, labels1, loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=3, frameon=False)

# Synchronize the two x-axes by matching the limits if needed
# ax2.set_xlim(ax3.get_xlim())
y_ref = 11.431576403321566
ax1.axhline(y=y_ref, color=rconfig.get_hex_code('grey', 150), linestyle='--')

# Annotate "Reference PVT" on the right of the plot but inside the plot area
ax1.annotate("Reference PVT", xy=(0.98, y_ref-0.5), xycoords=('axes fraction', 'data'),
             xytext=(0, 0), textcoords='offset points', fontsize=10, ha='right', va='center')

# ax1.set_xlim(0, 50)
# ax1.set_ylim(0,14)

y1_min, y1_max = ax1.get_ylim()  # Get limits of the primary Y-axis
ax2.set_ylim((y1_min-y_ref)/y_ref, (y1_max-y_ref)/y_ref)  # Apply the same transformation to match the secondary Y-axis scale
ax2.yaxis.set_major_formatter(FuncFormatter(to_percent))

fig.show()

In [ ]:
1.3*y_ref

In [ ]:
((harp_dict[(0.01, 68)]['X'][0][1] - 3*harp_dict[(0.01, 68)]['X'][0][3]) + (harp_dict[(0.01, 76)]['X'][0][1] - 3*harp_dict[(0.01, 76)]['X'][0][3]))/2

In [143]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_TH\TH-3-fig7.png', dpi=600)

In [ ]:
# harp_dict[(0.03, 12)]['df_res'][['h_fluid','T_fluid_mean','T_tube_mean','Qdot_absfin_back_conv']]
# harp_dict[(0.001, 12+8*26)]['df_res'][['h_fluid','T_fluid_mean','T_tube_mean','Qdot_absfin_back_conv']]

### Add fins of type 0

In [ ]:
L_f0_range = np.arange(0.005, 0.1, 0.005)
N_f0_range = np.array([30,40,50,60,70,80,90,100,110,120,130,140])
f0_dict = {}

for _, N_f0 in tqdm(enumerate(N_f0_range), total=len(N_f0_range)):

    # for i,L_f0 in tqdm(enumerate(L_f0_range), total=len(L_f0_range)):
    for i, L_f0 in enumerate(L_f0_range):

        panelSpecs_local = copy.deepcopy(panelSpecs)
        hyp_local = copy.deepcopy(hyp)
        steadyStateConditions_df_local = copy.deepcopy(steadyStateConditions_df)

        panelSpecs_local['part2']['Heta'] = L_f0
        panelSpecs_local['part2']['N_ail'] = N_f0

        panelSpecs_local['part2']['lambda_fin'] = 226
        panelSpecs_local['part2']['eps_fin'] = 0.09

        panelSpecs_local['part2']['D'] = panelSpecs_local['part2']['w_abs'] / panelSpecs_local['part2']['N_ail']

        panelSpecs_local['part2']['fin_0'] = 1
        panelSpecs_local['part2']['N_f0'] = panelSpecs_local['part2']['N_ail']
        panelSpecs_local['part2']['L_f0'] = panelSpecs_local['part2']['Heta']
        panelSpecs_local['part2']['delta_f0'] = panelSpecs_local['part2']['Dext_tube']
        panelSpecs_local['part2']['delta_f0_int'] = panelSpecs_local['part2']['p_ext_tube'] / 2

        # pr.update_panelSpecs_N_tubes(panelSpecs_local)
        pr.update_panelSpecs_after_modif(panelSpecs_local)

        df_res, list_res = ty.simu_steadyStateConditions(panelSpecs_local, hyp_local, steadyStateConditions_df_local)

        tab_mat,tab_res,X = nm.find_SK(df_res,dic_names,columns_list=[0,1,0,1,0,0,0,0,0],options=[],IAM=False,LSM='nnls')

        f0_dict[(N_f0,L_f0)] = {'panelSpecs' : copy.deepcopy(panelSpecs_local), 
                                'df_res' : copy.deepcopy(df_res),
                                'list_res' : copy.deepcopy(list_res),
                                'X' : copy.deepcopy(X)}

In [ ]:
ty.recap_residuals_steadyStateConditions(panelSpecs, f0_dict[(60,0.02)]['df_res'], f0_dict[(60,0.02)]['list_res'])

In [ ]:
f0_dict[(30,0.02)]['df_res'][['h_f0', 'Qdot_tube_back', 'Qdot_tube_back_rad', 'Qdot_absfin_back_conv', 'Qdot_f0', 'Qdot_tube_fluid']]

In [ ]:
f0_dict[(60,0.02)]['df_res'][['h_f0', 'Qdot_tube_back','Qdot_tube_back_rad', 'Qdot_absfin_back_conv', 'Qdot_f0','Qdot_tube_fluid']]

In [12]:
dp.write_pickle(storage_fp, "V4.NI_meander_calibrated_ssCforStudy_f0_parametric_Nf0_Lf0", f0_dict)

In [79]:
L_f0_range = np.arange(0.005, 0.1, 0.005)
N_f0_range = np.array([30,40,50,60,70,80,90,100,110,120,130,140])
f0_dict = dp.load_pickle(storage_fp, "V4.NI_meander_calibrated_ssCforStudy_f0_parametric_Nf0_Lf0")

### PLOT L_F0 and S_f0

In [ ]:
y_ref*1.1
print(N_range)
meander_b1[24-4]

In [ ]:
# Assuming rconfig and other variables are defined, here's the core part:
fig, ax1 = plt.subplots(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))

y_ref = 11.431576403321566

# Main Y-axis plot (ax1)
N_range = np.arange(4, 35, 1)
X0 = [X_list[i][0] for i in range(len(X_list))]  # Assuming X_list is already defined.

meander_b1 = [X0[i][1] - 3 * X0[i][3] for i in range(len(X0))]
ax1.plot(N_range, meander_b1, label='No fin', 
         color=colors[0], linestyle=linestyles[0], linewidth=1.5)

ax1.set_xlabel('Number of tubes')
ax1.set_ylabel(r"$b_{1} = A_1(0) \quad [W / m^{2} K]$")
ax1.set_xlim(0, 35)

# Create the secondary Y-axis (ax2) on the right
ax2 = ax1.twinx()

# Applying a simple transformation to the Y values for the secondary Y-axis.
# In this case, let's assume a linear transformation (e.g., multiplying by 2).
b1_transformed = (np.array(meander_b1) - y_ref)/y_ref
N_range = np.arange(4, 35, 1)
ax2.plot(N_range, b1_transformed, label=' ', linewidth=0.)

ax2.set_ylabel('Relative increase in $b_{1}$ vs. reference')
ax2.set_xlim(0, 35)

# Customizing and plotting the secondary X-axis (ax3) as you already have:
ax3 = ax1.twiny()
for i, N_f0 in enumerate([30, 60, 90, 120]):
    b1_list = [X[0][1] - 3*X[0][3] for X in [f0_dict[(N_f0, L_f0)]['X'] for L_f0 in L_f0_range]]
    ax3.scatter(L_f0_range * 1000, b1_list, label=f"$S_{{f0}} = {round((panelSpecs['part2']['w_abs'] / N_f0) * 1000, 1)} mm$", 
                marker=markers[i + 1], color=colors[i + 1])

ax3.set_xlabel("Fin height $L_{f0}$ [mm]")
ax3.set_xlim(0, 70)

# Adding the grid
ax1.grid(axis='x', linewidth=0.5, color=rconfig.get_hex_code('grey', 70))
ax2.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

# Combine the legends of both Y-axes
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()

# Plot combined legend
fig.legend(lines1 + lines2 + lines3, labels1 + labels2 + labels3, loc='upper center', 
           bbox_to_anchor=(0.5, 0.), ncol=3, frameon=False)

# Add a reference line
ax1.axhline(y=y_ref, color=rconfig.get_hex_code('grey', 150), linestyle='--')

# Annotate the reference line
ax1.annotate("Reference PVT", xy=(0.98, y_ref - 0.5), xycoords=('axes fraction', 'data'),
             xytext=(0, 0), textcoords='offset points', fontsize=10, ha='right', va='center')

y1_min, y1_max = ax1.get_ylim()  # Get limits of the primary Y-axis
ax2.set_ylim((y1_min-y_ref)/y_ref, (y1_max-y_ref)/y_ref)  # Apply the same transformation to match the secondary Y-axis scale
ax2.yaxis.set_major_formatter(FuncFormatter(to_percent))

# Display the figure
plt.show()

In [ ]:
f0_dict.keys()

In [ ]:
part2Specs = f0_dict[(120, 0.040)]['panelSpecs']['part2']

In [117]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_TH\TH-3-fig5.png', dpi=600)

### Add fins of type 1

In [ ]:
N_f1_range = np.array([30,40,50,60,70,80,90,100,110,120,130,140])
# D_tube_range = np.array([0.01, 0.02, 0.03, 0.04, 0.05])

f1_dict = {}

for _, N_f1 in tqdm(enumerate(N_f1_range), total=len(N_f1_range)):

    H_tube = 0.008

    X_list = []
    df_res_list = []
    list_res_list = []

    # for i,L_f0 in tqdm(enumerate(L_f0_range), total=len(L_f0_range)):
    # for i, L_f1 in enumerate(L_f1_range):

    panelSpecs_local = copy.deepcopy(panelSpecs)
    hyp_local = copy.deepcopy(hyp)
    steadyStateConditions_df_local = copy.deepcopy(steadyStateConditions_df)

    panelSpecs_local['part2']['H_tube'] = H_tube
    panelSpecs_local['part2']['w_tube'] = H_tube

    pr.update_panelSpecs_after_modif(panelSpecs_local)

    height = panelSpecs_local['part2']['Dext_tube']

    panelSpecs_local['part2']['Heta'] = height
    panelSpecs_local['part2']['N_ail'] = N_f1

    panelSpecs_local['part2']['lambda_fin'] = 226
    panelSpecs_local['part2']['eps_fin'] = 0.09

    panelSpecs_local['part2']['D'] = panelSpecs_local['part2']['w_abs'] / panelSpecs_local['part2']['N_ail']

    panelSpecs_local['part2']['fin_1'] = 1
    panelSpecs_local['part2']['coeff_f1'] = 1.
    panelSpecs_local['part2']['N_f1'] = panelSpecs_local['part2']['N_ail']
    panelSpecs_local['part2']['L_f1'] = panelSpecs_local['part2']['delta']
    panelSpecs_local['part2']['D_f1'] = panelSpecs_local['part2']['D']
    panelSpecs_local['part2']['delta_f1'] = panelSpecs_local['part2']['Dext_tube']
    panelSpecs_local['part2']['delta_f1_int'] = panelSpecs_local['part2']['p_ext_tube'] / 2

    # pr.update_panelSpecs_N_tubes(panelSpecs_local)
    pr.update_panelSpecs_after_modif(panelSpecs_local)

    df_res, list_res = ty.simu_steadyStateConditions(panelSpecs_local, hyp_local, steadyStateConditions_df_local)

    tab_mat,tab_res,X = nm.find_SK(df_res,dic_names,columns_list=[0,1,0,1,0,0,0,0,0],options=[],IAM=False,LSM='nnls')

    f1_dict[N_f1] = {'df_res' : copy.deepcopy(df_res), 'list_res' : copy.deepcopy(list_res), 'X' : copy.deepcopy(X)}

In [19]:
dp.write_pickle(storage_fp, "V4.NI_meander_calibrated_ssCforStudy_f1_parametric_Nf1", f1_dict)

In [70]:
N_f1_range = np.array([30,40,50,60,70,80,90,100,110,120,130,140])
f1_dict = dp.load_pickle(storage_fp, "V4.NI_meander_calibrated_ssCforStudy_f1_parametric_Nf1")

### Add fins of type 2

In [ ]:
N_f2_range = np.array([30,40,50,60,70,80,90,100,110,120,130,140])
# D_tube_range = np.array([0.01, 0.02, 0.03, 0.04, 0.05])

f2_dict = {}

for _, N_f2 in tqdm(enumerate(N_f2_range), total=len(N_f2_range)):

    H_tube = 0.008

    X_list = []
    df_res_list = []
    list_res_list = []

    panelSpecs_local = copy.deepcopy(panelSpecs)
    hyp_local = copy.deepcopy(hyp)
    steadyStateConditions_df_local = copy.deepcopy(steadyStateConditions_df)

    panelSpecs_local['part2']['H_tube'] = H_tube
    panelSpecs_local['part2']['w_tube'] = H_tube

    pr.update_panelSpecs_after_modif(panelSpecs_local)

    height = panelSpecs_local['part2']['Dext_tube']

    panelSpecs_local['part2']['Heta'] = height
    panelSpecs_local['part2']['N_ail'] = N_f2

    panelSpecs_local['part2']['lambda_fin'] = 226
    panelSpecs_local['part2']['eps_fin'] = 0.09

    panelSpecs_local['part2']['D'] = panelSpecs_local['part2']['w_abs'] / panelSpecs_local['part2']['N_ail']

    panelSpecs_local['part2']['fin_2'] = 1

    panelSpecs_local['part2']['N_f2'] = panelSpecs_local['part2']['N_ail']
    panelSpecs_local['part2']['L_f2'] = panelSpecs_local['part2']['delta']
    panelSpecs_local['part2']['D_f2'] = panelSpecs_local['part2']['D']
    panelSpecs_local['part2']['delta_f2'] = panelSpecs_local['part2']['Dext_tube']

    # pr.update_panelSpecs_N_tubes(panelSpecs_local)
    pr.update_panelSpecs_after_modif(panelSpecs_local)

    df_res, list_res = ty.simu_steadyStateConditions(panelSpecs_local, hyp_local, steadyStateConditions_df_local)

    tab_mat,tab_res,X = nm.find_SK(df_res,dic_names,columns_list=[0,1,0,1,0,0,0,0,0],options=[],IAM=False,LSM='nnls')

    f2_dict[N_f2] = {'df_res' : copy.deepcopy(df_res), 'list_res' : copy.deepcopy(list_res), 'X' : copy.deepcopy(X)}

In [28]:
dp.write_pickle(storage_fp, "V4.NI_meander_calibrated_ssCforStudy_f2_parametric_Nf2", f2_dict)

In [69]:
N_f2_range = np.array([30,40,50,60,70,80,90,100,110,120,130,140])
f2_dict = dp.load_pickle(storage_fp, "V4.NI_meander_calibrated_ssCforStudy_f2_parametric_Nf2")

In [ ]:
f1_dict[30]['df_res'][['h_f1', 'Qdot_tube_back', 'Qdot_tube_back_rad', 'Qdot_absfin_back_conv', 'Qdot_f1', 'Qdot_tube_fluid']]

In [ ]:
f2_dict[30]['df_res'][['h_f2', 'Qdot_tube_back', 'Qdot_tube_back_rad', 'Qdot_absfin_back_conv', 'Qdot_f2', 'Qdot_tube_fluid']]

### PLOT- FIN 1 and FIN 2

In [ ]:
colors = ['olive', 'green', 'blue', 'orange', 'red']
colors_lighter = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd', 'X', 'v', '^']

# Main plot: plotting L_f0_range on x-axis
fig, ax1 = plt.subplots(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))

# Set the new x-axis values for the number of tubes (assuming you have this data)

X_list = Ntubes_dict['X_list']
X0 = [X_list[i][0] for i in range(len(X_list))]

N_range = np.arange(4,35,1)
meander_b1 = [X0[i][1] - 3*X0[i][3]  for i in range(len(X0))]
ax1.plot(N_range, meander_b1, label = 'No fin', color=colors[0], linestyle=linestyles[0], linewidth = 1.5)

# ax1.plot([],[], color = rconfig.get_hex_code('grey', 150), linestyle = '--', label='Reference PVT')
# ax1.plot([],[], linewidth = 0., label=' ')

ax2 = ax1.twinx()

b1_transformed = (np.array(meander_b1) - y_ref)/y_ref
N_range = np.arange(4, 35, 1)
ax2.plot(N_range, b1_transformed, linewidth=0.)

ax2.set_ylabel('Relative increase in $b_{1}$ vs. reference')

ax1.set_xlabel('Number of tubes')
ax1.set_ylabel(r"$b_{1} = A_1(0) \quad [W / m^{2} K]$")

# Create a secondary x-axis
ax3 = ax1.twiny()

S_range = [round((panelSpecs['part2']['w_abs']/N_f)*1000,1) for N_f in N_f1_range]

f1_b1_list = [f1_dict[N_f1]['X'][0][1] - 3*f1_dict[N_f1]['X'][0][3] for N_f1 in N_f1_range]
f2_b1_list = [f2_dict[N_f]['X'][0][1] - 3*f2_dict[N_f]['X'][0][3] for N_f in N_f2_range]

ax3.scatter(S_range, f1_b1_list, label = 'Fin 1', marker = markers[1], color=colors[1])
ax3.scatter(S_range, f2_b1_list, label = 'Fin 2', marker = markers[3], color=colors[3])

# Set labels and legend for the first x-axis
ax3.set_xlabel("Fin spacing $S_{f}$ [mm]")

ax1.set_xlim(5,35)
ax3.set_xlim(5,35)

ax1.grid(axis='x', linewidth=0.5, color=rconfig.get_hex_code('grey', 70))
ax2.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

# Combine the legends of both axes
lines1, labels1 = ax1.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()

ax1.set_ylim(10,17)
ax3.set_ylim(10,17)

# Plot combined legend below the x-axis
fig.legend(lines1 + lines3, labels1 + labels3, loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=3, frameon=False)

# Synchronize the two x-axes by matching the limits if needed
# ax2.set_xlim(ax3.get_xlim())
y_ref = 11.431576403321566
plt.axhline(y=y_ref, color=rconfig.get_hex_code('grey', 150), linestyle='--')

# Annotate "Reference PVT" on the right of the plot but inside the plot area
ax1.annotate("Reference PVT", xy=(0.98, y_ref-0.5), xycoords=('axes fraction', 'data'),
             xytext=(0, 0), textcoords='offset points', fontsize=10, ha='right', va='center')

y1_min, y1_max = ax1.get_ylim()  # Get limits of the primary Y-axis
ax2.set_ylim((y1_min-y_ref)/y_ref, (y1_max-y_ref)/y_ref)  # Apply the same transformation to match the secondary Y-axis scale
ax2.yaxis.set_major_formatter(FuncFormatter(to_percent))

fig.show()

In [139]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_TH\TH-3-fig6.png', dpi=600)

In [ ]:
colors = ['olive', 'green', 'blue', 'orange', 'red']
colors_lighter = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd', 'X', 'v', '^']

# Main plot: plotting L_f0_range on x-axis
fig, ax3 = plt.subplots()

for i, N_f1 in enumerate([30, 60, 90, 120]):
    df_res = f1_dict[N_f1]['df_res']
    b1_list = [f1_dict[N_f1]['X'][0][1] - 3*f1_dict[N_f1]['X'][0][3] for D_tube in D_tube_range]

    ax3.scatter(D_tube_range, b1_list, 
                # label=f"$S_{{f0}} = {round((panelSpecs['part2']['w_abs']/N_f1)*1000,1)} mm$",
                label = f"$N_{{f1}} = {N_f1}$",
                marker = markers[i+1],
                color=colors[i+1]
                )
    
ax3.legend()

In [ ]:
for N_f0 in N_f0_range:
    Nf0_dict[N_f0]['list_res_list']

In [ ]:
ty.recap_residuals_steadyStateConditions(panelSpecs_local, f1_dict[(30,0.01)]['df_res'], f1_dict[(30,0.01)]['list_res'])

In [ ]:
Nf0_dict

In [ ]:
panelSpecs['part2']['w_abs'] / 100

In [ ]:
panelSpecs_local['part2']['view_factor_tube']

In [ ]:
# X_list = []

# for i,N in tqdm(enumerate(N_range)):

#     tab_mat,tab_res,X = nm.find_SK(df_res_list[i],dic_names,columns_list=[1,1,0,1,0,0,1,0,0],options=[],IAM=False,LSM='nnls')
#     X_list.append(X)

In [ ]:
for i, N_f0 in enumerate(N_f0_range):

    df_res_list = Nf0_dict[N_f0]['df_res_list']
    X_list = Nf0_dict[N_f0]['X_list']
    X0 = [X_list[i][0] for i in range(len(X_list))]

    plt.scatter(L_f0_range,[X0[i][1] - 3*X0[i][3]  for i in range(len(X0))], label = f'N_f0 = {N_f0}')
# plt.xlim(0,32)

plt.legend()

In [ ]:
L_f0_range[3]

In [ ]:
ty.recap_residuals_steadyStateConditions(Nf0_dict[100]['df_res_list'][3]

In [ ]:
Nf0_dict[30]['df_res_list'][3]['h_back']

In [ ]:
df_res_list[-1][['T_fluid_in','T_fluid_out', 'T_amb']]

In [ ]:
plt.scatter(N_range,[X0[i][0] for i in range(len(X0))])